In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers import logging
import torch

D:\venvs\venv_310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(torch.version.cuda)


Using device: cuda
11.8


In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"  # Or another GPTQ model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

logging.set_verbosity_info()  # or logging.set_verbosity_debug()

In [ ]:
device_map = {"": "cuda:0"}  # If you want to explicitly assign the whole model to GPU 0

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    revision="main"
)


loading configuration file config.json from cache at C:\Users\Farshad\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.1-GPTQ\snapshots\6ae1e4ae2cfbaf107c705ed722ec243b4f88014d\config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bits": 4,
    "damp_percent": 0.1,
    "desc_act": true,
    "group_size": 128,
    "model_file_base_name": "model",
    "model_name_or_path": null,
    "quant_method": "gptq",
    "sym": true,
    "true_sequential": true
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4


WARN  Feature `utils/Perplexity` requires python GIL. Feature is currently skipped/disabled.                           
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.                          
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                                                  


loading weights file model.safetensors from cache at C:\Users\Farshad\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.1-GPTQ\snapshots\6ae1e4ae2cfbaf107c705ed722ec243b4f88014d\model.safetensors
Instantiating MistralForCausalLM model under default dtype torch.float16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0
}



INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`                                                     


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_pr

INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                                        
INFO  Format: Converting GPTQ v1 to v2                                                                                 
INFO  Format: Conversion complete: 0.031057357788085938s                                                               
WARN  Feature `Torch Compile` requires python GIL. Feature is currently skipped/disabled.                              
INFO  Optimize: `TorchQuantLinear` compilation triggered.                                                              


In [10]:
model = torch.compile(model)
model.eval()

OptimizedModule(
  (_orig_mod): MistralForCausalLM(
    (model): MistralModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x MistralDecoderLayer(
          (self_attn): MistralAttention(
            (k_proj): TorchQuantLinear()
            (o_proj): TorchQuantLinear()
            (q_proj): TorchQuantLinear()
            (v_proj): TorchQuantLinear()
          )
          (mlp): MistralMLP(
            (act_fn): SiLU()
            (down_proj): TorchQuantLinear()
            (gate_proj): TorchQuantLinear()
            (up_proj): TorchQuantLinear()
          )
          (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        )
      )
      (norm): MistralRMSNorm((4096,), eps=1e-05)
      (rotary_emb): MistralRotaryEmbedding()
    )
    (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
  )
)

In [ ]:
torch.set_float32_matmul_precision("high")

# Text streamer
streamer = TextStreamer(tokenizer)

# Prompt
prompt = "Explain quantum entanglement in simple terms."
prompt = "Explain how genetics and AI can bring about breakthroughs. explain very brief"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,          # Disable sampling, use greedy decoding
        num_beams=1,              # No beam search
        temperature=1.0,
        top_k=0,
        top_p=1.0,
        repetition_penalty=1.0,
        streamer=streamer
    )

# If not using streamer
# print(tokenizer.decode(output[0], skip_special_tokens=True))

D:\venvs\venv_310\lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Explain how genetics and AI can bring about breakthroughs. explain very brief history of genetics and AI.

Genetics and AI can bring about breakthroughs in various fields by providing new insights and tools for understanding and manipulating complex systems. Genetics can help us understand the underlying mechanisms of inheritance and disease, while AI can help us analyze and interpret large amounts of data.

The history of genetics dates back to the early 20th century, when scientists such as Gregor Mendel and Thomas Hunt Morgan began studying the patterns of inherit
